***starting the Project Customer Support ticket rating prediction***

In [1]:
!pip install nltk
!pip install gensim

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)

   ---------------------------------------- 0/3 [regex]
   ------------- -------------------------- 1/3 [click]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   ---

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


In [2]:
import  numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Loading the dataset
df = pd.read_csv("customer_support_tickets.csv")

In [5]:
# View first few rows of the dataset
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [8]:
# columns that are useful for our model 
# 'Product Purchased','Customer Gender','Ticket Type','Ticket Subject','Ticket Description',
    # 'Ticket Status',
    # 'Ticket Priority',
    # 'Resolution',
    # 'First Response Time',
    # 'Time to Resolution',
    # 'Customer Satisfaction Rating'

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

In [10]:
df.describe()

,Ticket ID,Customer Age,Customer Satisfaction Rating
count,8469.000000,8469.000000,2769.000000
mean,4235.000000,44.026804,2.991333
std,2444.934048,15.296112,1.407016
min,1.000000,18.000000,1.000000
25%,2118.000000,31.000000,2.000000
50%,4235.000000,44.000000,3.000000
75%,6352.000000,57.000000,4.000000
max,8469.000000,70.000000,5.000000


In [11]:
df.isnull().sum()

Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64

In [12]:
# we can see the there are so many null values in the dataset. so ther 
# is the need to clean the dataset 

In [13]:
cols_to_use = [
    'Product Purchased',
    'Customer Gender',
    'Ticket Type',
    'Ticket Subject',
    'Ticket Description',
    'Ticket Status',
    'Ticket Priority',
    'Resolution',
    'First Response Time',
    'Time to Resolution',
    'Customer Satisfaction Rating'
]

data = df[cols_to_use].copy()


In [14]:
# Dropping missing ratings so that we can train a model based on the remaining data
data = data.dropna(subset=["Customer Satisfaction Rating"])

# Convert rating to int ( for the sake of the classication)
data["Customer Satisfaction Rating"] = data["Customer Satisfaction Rating"].astype(int)


In [15]:
# we have some date time columns so we need to convert them to datetime format and extract their features

for col in ["First Response Time", "Time to Resolution"]:
    data[col] = pd.to_datetime(data[col], errors="coerce")

# Extract month and day , no need of the year 
data["First Response Month"] = data["First Response Time"].dt.month
data["First Response Day"] = data["First Response Time"].dt.day
data["Resolution Month"] = data["Time to Resolution"].dt.month
data["Resolution Day"] = data["Time to Resolution"].dt.day

# Drop original datetime columns as this is not for use 
data.drop(columns=["First Response Time", "Time to Resolution"], inplace=True)

In [16]:
data.head()

,Product Purchased,Customer Gender,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Ticket Priority,Resolution,Customer Satisfaction Rating,First Response Month,First Response Day,Resolution Month,Resolution Day
2,Dell XPS,Other,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Low,Case maybe show recently my computer follow.,3,6,1,6,1
3,Microsoft Office,Female,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Low,Try capital clearly never color toward story.,3,6,1,6,1
4,Autodesk AutoCAD,Female,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,Low,West decision evidence bit.,1,6,1,6,1
10,Nintendo Switch,Male,Cancellation request,Data loss,I'm having an issue with the {product_purchase...,Closed,High,Measure tonight surface feel forward.,1,6,1,5,31
11,Microsoft Xbox Controller,Male,Product inquiry,Software bug,I'm having an issue with the {product_purchase...,Closed,High,Measure there house management pick knowledge ...,1,6,1,6,1


In [17]:
data.isnull().sum()

Product Purchased               0
Customer Gender                 0
Ticket Type                     0
Ticket Subject                  0
Ticket Description              0
Ticket Status                   0
Ticket Priority                 0
Resolution                      0
Customer Satisfaction Rating    0
First Response Month            0
First Response Day              0
Resolution Month                0
Resolution Day                  0
dtype: int64

In [18]:
data.shape

(2769, 13)

In [20]:
# now start cleaning the text data - description and resolution
"""We will-
 Lowercase all text
  Remove digits
  Remove punctuation
  Tokenize
  Remove stopwords"""

'We will-\n Lowercase all text\n  Remove digits\n  Remove punctuation\n  Tokenize\n  Remove stopwords'

In [21]:
#  required libraries and word lists
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Set of English stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!







**TEXT CLEANING**


I am going to do create the function to remove - punctuation , stopwords , digits and html tags 
lowercasing is also important , tokenization will be done simultaneously 

In [23]:
def clean_and_tokenize(text):
    text = str(text).lower()                                # 1. Lowercase
    text = re.sub(r'<.*?>', '', text)                       # 2. Remove HTML tags
    text = re.sub(r'\d+', '', text)                         # 3. Remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))  # 4. Remove punctuation
    tokens = word_tokenize(text)                            # 5. Tokenize
    tokens = [word for word in tokens if word not in stop_words]      # 6. Remove stopwords
    return tokens


**Now apply cleaning to both the columns**

In [24]:
# Apply to Ticket Description and Resolution (these are the only text containing columns)
data["clean_desc"] = data["Ticket Description"].apply(clean_and_tokenize)
data["clean_res"] = data["Resolution"].astype(str).apply(clean_and_tokenize)

***Train the Word2vec for the combined corpus of the des+res***

we have to take the essence of both the columns and control the dimension as well to get to our result , and average word2vec will do the work for us --it will take the average of the embeddings of the words or tokens and if the the sentence is empty return the 0 vector for the dimension of the vector


In [ ]:
from gensim.models import Word2Vec

# Combining corpus of both for Word2Vec training
combined_tokens = data["clean_desc"].tolist() + data["clean_res"].tolist()

# Training model 
w2v_model = Word2Vec(sentences=combined_tokens, vector_size=100, window=5, min_count=1, workers=4, seed=42)


***NOW*** creating a function that will convert the tokens in the embeddings or vector and then we will take the average of the the vectors 

In [26]:
def avg_word2vec(tokens, model, size=100):
    # Create an empty list to store word vectors
    word_vectors = []

    # Loop through each word (token) in the tokens list
    for word in tokens:
        # If the word exists in the Word2Vec model's vocabulary then we will add it to the list
        if word in model.wv:
            word_vectors.append(model.wv[word])

    
    if len(word_vectors) > 0:
        avg_vector = np.mean(word_vectors, axis=0)
    else:
        avg_vector = np.zeros(size)

    return avg_vector
# this will do our work 


***NOW*** we will change the description and the resulution into the embeddings 



***BUT BUT BUT***

***descision*** pending on what to do 
we have 2 ways (add the embeddings of res+desc or not)
   add the embeddings of the description + resolution to reduce the dimension of the data but we have only 2700 rows in which we will split so i guess increasing dimension may help in data getting complex so that the ANN will work much better and for overfitting we have some techniques that can be used to reduce it 

In [27]:
# Create an empty list to store all average vectors
desc_vectors = []

# Loop through each row
for tokens in data["clean_desc"]:
    # Get average Word2Vec for that row's token list
    avg_vector = avg_word2vec(tokens, w2v_model)
    # Append the result
    desc_vectors.append(avg_vector)

# Convert the list of vectors into a DataFrame
desc_df = pd.DataFrame(desc_vectors, columns=[f"desc_vec_{i}" for i in range(100)])


# Same process for the resolution column
res_vectors = []

for tokens in data["clean_res"]:
    avg_vector = avg_word2vec(tokens, w2v_model)
    res_vectors.append(avg_vector)

res_df = pd.DataFrame(res_vectors, columns=[f"res_vec_{i}" for i in range(100)])
# Combine the original data with the new vector columns

***Merge These Vectors Back into Main Dataset***

In [28]:
# Concatenate the new vectors to original dataset
data = pd.concat([data.reset_index(drop=True), desc_df, res_df], axis=1)

# Drop original and temporary text columns , this is of no use now , haha
data.drop(columns=["Ticket Description", "Resolution", "clean_desc", "clean_res"], inplace=True)


In [29]:
# now lets check , how our data looks like

In [30]:
data.head()

,Product Purchased,Customer Gender,Ticket Type,Ticket Subject,Ticket Status,Ticket Priority,Customer Satisfaction Rating,First Response Month,First Response Day,Resolution Month,...,res_vec_90,res_vec_91,res_vec_92,res_vec_93,res_vec_94,res_vec_95,res_vec_96,res_vec_97,res_vec_98,res_vec_99
0,Dell XPS,Other,Technical issue,Network problem,Closed,Low,3,6,1,6,...,-0.370908,0.310422,0.120041,-0.118673,0.033079,-0.361517,0.178009,0.093716,-0.183325,-0.124308
1,Microsoft Office,Female,Billing inquiry,Account access,Closed,Low,3,6,1,6,...,-0.300256,0.305272,0.012953,-0.223197,0.163965,-0.156620,0.120185,0.289480,-0.111362,-0.105171
2,Autodesk AutoCAD,Female,Billing inquiry,Data loss,Closed,Low,1,6,1,6,...,-0.333727,0.318661,0.076638,-0.209621,0.125945,-0.169298,0.160390,0.257023,-0.110265,-0.098607
3,Nintendo Switch,Male,Cancellation request,Data loss,Closed,High,1,6,1,5,...,-0.305083,0.287482,0.077222,-0.206290,0.127977,-0.161067,0.157802,0.249840,-0.089301,-0.077484
4,Microsoft Xbox Controller,Male,Product inquiry,Software bug,Closed,High,1,6,1,6,...,-0.264081,0.247066,0.065887,-0.169593,0.109208,-0.134181,0.143271,0.217789,-0.094058,-0.061470


In [31]:
data.shape

(2769, 211)

***now** we still have to do label encoding to the remaining columns 

In [32]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# List of categorical columns to encode inour data
cat_cols = [
    'Product Purchased', 'Customer Gender', 'Ticket Type',
    'Ticket Subject', 'Ticket Status', 'Ticket Priority'
]

# this is the label encoding part
for col in cat_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])


In [34]:
data.head()

,Product Purchased,Customer Gender,Ticket Type,Ticket Subject,Ticket Status,Ticket Priority,Customer Satisfaction Rating,First Response Month,First Response Day,Resolution Month,...,res_vec_90,res_vec_91,res_vec_92,res_vec_93,res_vec_94,res_vec_95,res_vec_96,res_vec_97,res_vec_98,res_vec_99
0,10,2,4,8,0,2,3,6,1,6,...,-0.370908,0.310422,0.120041,-0.118673,0.033079,-0.361517,0.178009,0.093716,-0.183325,-0.124308
1,25,0,0,0,0,2,3,6,1,6,...,-0.300256,0.305272,0.012953,-0.223197,0.163965,-0.156620,0.120185,0.289480,-0.111362,-0.105171
2,5,0,0,3,0,2,1,6,1,6,...,-0.333727,0.318661,0.076638,-0.209621,0.125945,-0.169298,0.160390,0.257023,-0.110265,-0.098607
3,30,1,1,3,0,1,1,6,1,5,...,-0.305083,0.287482,0.077222,-0.206290,0.127977,-0.161067,0.157802,0.249840,-0.089301,-0.077484
4,27,1,2,15,0,1,1,6,1,6,...,-0.264081,0.247066,0.065887,-0.169593,0.109208,-0.134181,0.143271,0.217789,-0.094058,-0.061470


***Hurrey*** our preprocessing step is now done now its Modelling time , as we are working on the deep learning model so i switched to the high performance GPU training(this will help in accelerate the calculation)

***BUT*** before modelling we need to scale the input and output otherwise it can give us uneven answers 
***after scaling*** we have to split the data in the train(train+validation) , test .......some people uses test data as validation but i am not gonna do that 

In [35]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# Features = everything except the target column
X = data.drop(columns=["Customer Satisfaction Rating"])

# Target = classification labels (1 to 5) → shift to 0-indexed for keras
y_raw = data["Customer Satisfaction Rating"].values - 1  # convert from 1–5 to 0–4

# One-hot encode target (for multiclass classification)
y = to_categorical(y_raw, num_classes=5)


***Split into Train, Validation, and Test***

In [65]:
# First: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y_raw)

# From 80%, take 20% for validation → final: 64% train, 16% val, 20% test
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp.argmax(axis=1))
# Scale features is our next step

***We use StandardScaler to ensure ANN training stability.***

In [66]:
scaler = StandardScaler()

# Fit only on training
X_train_scaled = scaler.fit_transform(X_train)

# Transform val and test with same scaler
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


**Why We Don't Fit on All of X** 
If you fit StandardScaler on all the data, you're letting the model peek into the test set during training via: Mean &Standard deviation  , this prevents data leakage 

***ITS TIME FOR THE MODEL ARCHITECTURE TO BE SELECTED***

***BUT BUT BUT***

***I WANT TO IMPLEMENT KERAS TUNNER FOR HYPERPARAMETER TUNNING SO LETS DIVE INTO IT***

In [39]:
# LETS FIRST INSTALL THE KERAS TUNNER
%pip install keras-tuner


   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   ---------------------------------------- 2/2 [keras-tuner]

Note: you may need to restart the kernel to use updated packages.


importing the required layers for the ann architecture 

In [40]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


***some important things***
***things that gonna help me reduce the overfitting*** -> dropouts(drops the nodes while backpropagation) , Earlystopping(stop when there is no improvement),
batch normalization(for smooth training),

***optimizer***-> 
Adam - best forever(rmsprop can be used but i am gonna use adam)

***loss funtion***-> 
categorical_crossentropy -> we use this  in case of multiclass-classification

***epochs*** = 100 (conventionally used)

In [41]:
def build_model(hp):
    model = Sequential()
    
    # Input layer
    model.add(Dense(
        hp.Int('units_input', min_value=64, max_value=512, step=64),
        activation='relu',
        input_shape=(X_train_scaled.shape[1],)
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_input', 0.2, 0.5, step=0.1)))

    # Optional hidden layers
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(Dense(
            hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
            activation='relu'
        ))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1)))

    # Output layer
    model.add(Dense(5, activation='softmax'))

    # Compile with tunable learning rate
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('lr', 1e-4, 1e-2, sampling='log')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [47]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=100,                  # Number of hyperparameter sets to try
    executions_per_trial=1,        # Number of models to build per set (avg result)
    directory='tuning_dir',
    project_name='rating_prediction'
)


Reloading Tuner from tuning_dir\rating_prediction\tuner0.json


In [48]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',        # What to monitor? (usually 'val_loss' or 'val_accuracy')
    patience=10,               # How many epochs to wait before stopping after no improvement
    restore_best_weights=True,# Roll back to best model when training ends
    verbose=1                  # Print info when stopping
)


In [49]:
tuner.search(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)]
)


Trial 100 Complete [00h 00m 12s]
val_accuracy: 0.21218961477279663

Best val_accuracy So Far: 0.2505643367767334
Total elapsed time: 01h 23m 29s


In [50]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate on test set
test_loss, test_acc = best_model.evaluate(X_test_scaled, y_test)
print(f"Best Test Accuracy: {test_acc:.4f}")


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1788 - loss: 1.6853  
Best Test Accuracy: 0.1986


***RESULT***
the deep learninng algo is giving a accuracy score of 20%

In [51]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(objective='multi:softmax', num_class=5, eval_metric='mlogloss')
xgb.fit(X_train_scaled, y_train.argmax(axis=1))

# Predict
y_pred = xgb.predict(X_test_scaled)
print("Test Accuracy:", accuracy_score(y_test.argmax(axis=1), y_pred))


Test Accuracy: 0.2003610108303249


***CONCLUSION:***
   THE ANN and XGboost both are giving the accuracy of 20%

   ***why is so:-***
   1. the data is so less (major reason)
   2. wrong labelling 
   3. ANN and ML can never understand the true meaning of sequential data(because order of words is not kept)


   So its Futuristic solution is to Apply ***LSTM or Transformer***. 
   but that will also need the high amount of data to train and that is not available now
   

***SO this is my report for this projet***
                     -----***Tanish Jangir***